In [ ]:
# Before doing anything, we must runtime install some libraries into jupyterlite
%pip install -q nbformat plotly upstage-des>=0.4,<5

# Cashier Simulation

This is an example model built using UPSTAGE, where cashiers check customers through checkout lanes, help restock shelves, and have periodic breaks.

In [ ]:
import upstage_des.api as UP
from upstage_des.data_utils import create_table
from model.cashier_model import (
    Cashier,
    CheckoutLane,
    cashier_task_network,
    cashier_message_net,
    customer_spawner,
    manager_process,
    StoreBoss,
)
from model.helpers import to_step, doing_to_gantt
import pandas as pd
from datetime import datetime, timedelta

import plotly.express as px

## Sim Creation

The simulation is created inside the `EnvironmentContext`. Note that we are controlling the random seed, setting a start time other than 0, and that data gathering must happen in the context.

Normally you'd want to put most of this code inside a function:

```python
def build_sim(sim_data) -> SimObjects:
    ...

with UP.EnvironmentContext() as env:
    simobj = build_sim(some_data_source)
    env.run()
```

but for now we'll put everything in one place so you can see the machinery.

In [ ]:
with UP.EnvironmentContext(initial_time=8 * 60, random_seed=2881680) as env:
    UP.add_stage_variable("time_unit", "min")
    cashier = Cashier(
        name="Bob",
        scan_speed=1.0,
        time_until_break=160.0,
        breaks_until_done=4,
        debug_log=True,
    )
    cashier2 = Cashier(
        name="Ertha",
        scan_speed=1.0,
        time_until_break=160.0,
        breaks_until_done=4,
        debug_log=True,
    )
    lane_1 = CheckoutLane(name="Lane 1")
    lane_2 = CheckoutLane(name="Lane 2")
    boss = StoreBoss(lanes=[lane_1, lane_2])

    UP.add_stage_variable("boss", boss)

    for cash in [cashier, cashier2]:
        net = cashier_task_network.make_network()
        cash.add_task_network(net)
        cash.start_network_loop(net.name, "GoToWork")

        net = cashier_message_net.make_network()
        cash.add_task_network(net)
        cash.start_network_loop(net.name, "CashierMessages")

    customer_proc = customer_spawner(env, [lane_1, lane_2], max_wait=6.)
    _ = env.process(customer_proc)

    _ = env.process(manager_process(boss, [cashier, cashier2]))

    # Optional forcing of a restock before a break to test behavior
    # def _proc():
    #     yield env.timeout(8*60 + 10)
    #     yield cashier2.messages.put(["Restock"])
    # env.process(_proc())

    env.run(until=20 * 60)
    data, cols = create_table()

## Data Processing

Create a pandas dataframe of the data and convert the time to a datetime for use with Plotly.

In [ ]:
df = pd.DataFrame(data, columns=cols)
# convert time to a datetime
start = datetime(2025, 6, 10)
df["Time"] = df.Time.apply(lambda x: start + timedelta(minutes=x))

## Visualization

The data from UPSTAGE is not immediately formatted for some plot styles. See `model/helpers.py` for code that turns the raw data into more plotting friendly forms.

## Customer Queue Sizes

In [ ]:
for name, grp in df[(df["Entity Type"]=="CheckoutLane")].groupby("Entity Name"):
    stepped = to_step(
        [
            (row["Time"], row["Value"])
            for _, row in grp.iterrows()
            if row["State Name"] == "customer_queue"
        ],
        last_time=df.Time.max(),
    )
    dfgrp = pd.DataFrame(stepped, columns=["Time", "Customers Waiting"])
    fig = px.line(dfgrp, x='Time', y="Customers Waiting")
    fig.update_layout(title_text=f"{name} Data", )
    fig.show()

## Cashier Task Timeline

Using a `State` that is recording and stores a string is a great way to easily get activity data over time for your actors. This is highly recommended for quicker debugging and model verification.

In this model, `current_task` is the `State`, and we process the state date into a gantt chart form for Plotly.

In [ ]:
g1 = doing_to_gantt(df, "Bob", "current_task")
g2 = doing_to_gantt(df, "Ertha", "current_task")
g1["Cashier"] = "Bob"
g2["Cashier"] = "Ertha"
g = pd.concat((g1, g2))

In [ ]:
fig = px.timeline(
    g, facet_col="Cashier", x_start="Start", x_end="Finish", y="Task", color="Task",
    facet_col_wrap=1, facet_row_spacing=0.1, height=600)
fig.show()